In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import re
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.model_selection import cross_val_score


%matplotlib inline

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 61]
[nltk_data]     Connection refused>


False

In [3]:
from nltk.stem import SnowballStemmer  
snowball_stemmer = SnowballStemmer("english")  
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [4]:
def print_plot(index):
    example = df[df.index == index][[test_slice, 'tag']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Genre:', example[1])

In [5]:
import string

df1 = pd.read_csv('data/AS_smell_singleanswer.csv')
df2 = pd.read_csv('data/non_AS_smell_singleanswer_utf8.csv')
df = pd.concat([df1,df2])
df = df.dropna()
df['total'] = df['tittle'] + df['question'] + df['answer']
df = df.replace('string.punctuation',' ')
test_slice = 'total'

In [6]:
for i in range(395):
    text = df.iloc[i,6]
    remove = str.maketrans('','',string.punctuation) 
    df.iloc[i,6] = text.translate(remove)
#     tokens = nltk.word_tokenize(without_punctuation)

In [7]:
print_plot(12)

What would be a nice architecture so I can pass information of eventual problems to a higher layerI have a standard 3 layer project and my data layer either accesses the database or an API I would like to be able to show information to the user in case there are problems occurring in the data layer
I would like to show in the interface if there was any error some required info by the API was not set problems connecting to the databaseAPI or any other problem that could happen
Currently I would have an out string error parameter that would return the problem message But this smells bad and this will also require my business layer to have the same paramter which smells even worse
What would be a nice architecture so I can pass information of eventual problems to a higher layer 
ThanksThere could be two ways

Raising some custom exception
Raising event at database layer and handle it in business layer

Now while raising the eventexception at data layer you can pass the information which w

In [8]:
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
print(len(df))
print(len(df1))
print(len(df2))

395
208
187


# 验证函数

In [9]:
def evaluate_prediction(predictions, target, title="Confusion matrix"):
    print('accuracy %s' % accuracy_score(target, predictions))
    print('precision %s' % precision_score(target, predictions))
    print('recall %s' % recall_score(target, predictions))
    print('F1 %s' % f1_score(target, predictions))
    print("\n")
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(my_tags))
    target_names = my_tags
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data[test_slice])
    predictions = classifier.predict(data_features)
    target = data['tag']
    evaluate_prediction(predictions, target)
    print("\n")
def ten_cross_fold_predict(classifier, x, y):
    n = 5
    print('accuracy %s' % np.mean(cross_val_score(classifier, x, y, scoring='accuracy', cv=n)))
    print('precision %s' % np.mean(cross_val_score(classifier, x, y, scoring='precision', cv=n)))
    print('recall %s' % np.mean(cross_val_score(classifier, x, y, scoring='recall', cv=n)))
    print('F1 %s' % np.mean(cross_val_score(classifier, x, y, scoring='f1', cv=n)))
def most_influential_words(vectorizer, genre_index=0, num_words=10):
    features = vectorizer.get_feature_names()
    max_coef = sorted(enumerate(logreg.coef_[genre_index]), key=lambda x:x[1], reverse=True)
    return [features[x[0]] for x in max_coef[:num_words]]
def remove(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, '', text)

# word embedding

## Averaging word vectors

In [10]:
wv = gensim.models.KeyedVectors.load_word2vec_format( "data/GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

2020-04-24 17:47:11,216 : INFO : loading projection weights from data/GoogleNews-vectors-negative300.bin.gz
2020-04-24 17:49:14,809 : INFO : loaded (3000000, 300) matrix from data/GoogleNews-vectors-negative300.bin.gz
2020-04-24 17:49:14,825 : INFO : precomputing L2-norms of word weight vectors


In [11]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.layer1_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [12]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 3:
                continue
#             if word in stopwords.words('english'):
#                 continue
            tokens.append(word)
    return tokens

In [13]:
def stem_lemma(text):
    words = []
    for doc in text:
        for word in doc:
            stem_word = snowball_stemmer.stem(word)
            lemma_word = lmtzr.lemmatize(stem_word)
            words.append(lemma_word)
    return words

In [14]:
# stem_lemma(train_tokenized)
# train_tokenized

In [15]:
test_tokenized = test_data.apply(lambda r: w2v_tokenize_text(r[test_slice]), axis=1).values
train_tokenized = train_data.apply(lambda r: w2v_tokenize_text(r[test_slice]), axis=1).values

In [16]:
%%time
X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

<ipython-input-11-b93e2c1d24eb>:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  mean.append(wv.syn0norm[wv.vocab[word].index])


CPU times: user 636 ms, sys: 721 ms, total: 1.36 s
Wall time: 2.81 s


In [17]:
# 10_fold
df_tokenized = df.apply(lambda r: w2v_tokenize_text(r[test_slice]), axis=1).values
df_word_average = word_averaging_list(wv,df_tokenized)

svc_10 = svm.SVC(gamma='scale')
rf_10 = RandomForestClassifier(n_estimators=10)
logreg_10 = linear_model.LogisticRegression(n_jobs=1, C=1e5)
knn_10 = KNeighborsClassifier(n_neighbors=3, n_jobs=1, algorithm='brute', metric='cosine' )
ten_cross_fold_predict(logreg_10, df_word_average, df['tag'])
ten_cross_fold_predict(rf_10, df_word_average, df['tag'])
ten_cross_fold_predict(svc_10, df_word_average, df['tag'])
ten_cross_fold_predict(knn_10, df_word_average, df['tag'])


<ipython-input-11-b93e2c1d24eb>:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  mean.append(wv.syn0norm[wv.vocab[word].index])
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("D

accuracy 0.5644060370009737


/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed 

precision 0.6010987042514137


/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed 

recall 0.562137049941928


/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed 

F1 0.570628706332031
accuracy 0.554439305420318
precision 0.5342886970290772
recall 0.5530778164924506
F1 0.5401724543180924
accuracy 0.5743743914313535
precision 0.560020929774451
recall 0.8891986062717769
F1 0.6870255256329727
accuracy 0.5467786432976307
precision 0.5652203320352531
recall 0.6013937282229965
F1 0.5776668372503346


In [18]:
# logistic

logreg = linear_model.LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train_data['tag'])
predictions = logreg.predict(X_test_word_average)
evaluate_prediction(predictions, test_data.tag)

accuracy 0.475
precision 0.5
recall 0.47619047619047616
F1 0.4878048780487805




/Users/lf/PycharmProjects/ASsmell/venv/lib/python3.8/site-packages/sklearn/linear_model/logistic.py:430: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "


In [19]:
# SVM
from sklearn import svm
svc = svm.SVC(gamma='scale')
svc = svc.fit(X_train_word_average, train_data['tag'])
predictions = svc.predict(X_test_word_average)
evaluate_prediction(predictions, test_data.tag)

#random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf = rf.fit(X_train_word_average, train_data['tag'])
predictions = rf.predict(X_test_word_average)
evaluate_prediction(predictions, test_data.tag)

# knn
knn_naive_dv = KNeighborsClassifier(n_neighbors=3, n_jobs=1, algorithm='brute', metric='cosine' )
knn_naive_dv.fit(X_train_word_average, train_data.tag)
predicted = knn_naive_dv.predict(X_test_word_average)
evaluate_prediction(predicted, test_data.tag)


# 依次为SVM，RandomForest，KNN

accuracy 0.65
precision 0.6129032258064516
recall 0.9047619047619048
F1 0.7307692307692307


accuracy 0.55
precision 0.5789473684210527
recall 0.5238095238095238
F1 0.5500000000000002


accuracy 0.575
precision 0.5833333333333334
recall 0.6666666666666666
F1 0.6222222222222222




In [20]:
test_data.iloc()[56][test_slice]

IndexError: single positional indexer is out-of-bounds

In [ ]:
wv.most_similar(positive=[X_test_word_average[56]], restrict_vocab=100000, topn=30)[0:20]

# doc2vec

In [ ]:
sentences = gensim.models.doc2vec.TaggedLineDocument('data/AS_smell_singleanswer.csv')

In [ ]:
model = gensim.models.Doc2Vec(sentences, size = 100, window = 5)
model.save('AS_text_model.txt')
print len(model.docvecs)